In [1]:
%%configure -f
{"executorCores": 4, "executorMemory": "8096M", "conf": {"spark.default.parallelism": 1000}}

In [2]:
import pandas as pd

import gensim
from gensim.utils import tokenize
from gensim.parsing.preprocessing import remove_stopwords
import string
import numpy as np
from langdetect import detect
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType, ArrayType, DoubleType, MapType
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml import Pipeline
import smart_open

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1573125328205_0003,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df_csv = pd.read_csv(smart_open.open("s3://ai-data-lake-dev-eu-west-1/staging/peer/SIC_REV3.txt"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
code2desc = dict(zip(df_csv.Code, df_csv.Description))
code2parent = dict(zip(df_csv.Code, df_csv.Parent))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df = spark.read.load("s3://ai-data-lake-dev-eu-west-1/staging/peer/company_data_denormalized_clean")
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- entity_id: string (nullable = true)
 |-- vendor: string (nullable = true)
 |-- status: string (nullable = true)
 |-- id_capiq: integer (nullable = true)
 |-- id_bvd: string (nullable = true)
 |-- primary_bvd_node: string (nullable = true)
 |-- name: string (nullable = true)
 |-- business_description: string (nullable = true)
 |-- primary_sic_node: string (nullable = true)
 |-- primary_naics_node: string (nullable = true)
 |-- primary_nace_node: string (nullable = true)
 |-- has_listed_equity: string (nullable = true)
 |-- country_of_incorporation: string (nullable = true)
 |-- country_of_incorporation_iso: string (nullable = true)
 |-- operating_countries: string (nullable = true)
 |-- number_employees: integer (nullable = true)
 |-- ccc: double (nullable = true)
 |-- revenue: double (nullable = true)
 |-- total_revenue: double (nullable = true)
 |-- net_inc_incl_xo: double (nullable = true)
 |-- total_debt_repaid: double (nullable = true)
 |-- ebitda: double (nullable = true

In [6]:
def add_parent_code_desc(sic_code):
    if not sic_code or sic_code not in code2desc:
        return ""
    desc = code2desc[sic_code]
    id_parent = code2parent[sic_code]
    parent_desc = code2desc[id_parent]
    if desc != parent_desc:
        desc += " " + parent_desc
    return desc

add_parent_code_desc_udf = udf(add_parent_code_desc, StringType())
df = df.withColumn("enhanced_sic_desc", add_parent_code_desc_udf('primary_sic_node')) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
(df
 .write
 .parquet("s3://ai-data-lake-dev-eu-west-1/staging/peer/company_data_denormalized_clean_extra_sic_code", mode="overwrite")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# df_new = spark.read.load("s3://ai-data-lake-dev-eu-west-1/staging/peer/company_data_denormalized_deduped")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…